In [318]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import StandardScaler
from os import listdir
from os.path import isfile, join
import os
from sklearn import tree
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import math
import xgboost as xgb
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [319]:
# Set paths
p = os.path.abspath('..')
p1 = p + '/raw data 17-19/'
p2 = p + '/engineered features 17-19/'

# Get all file names.csv
onlyfiles = [f for f in listdir(p1) if isfile(join(p1, f))]

raw_df=[]
tech_df=[]
for f, stock_num in zip([onlyfiles[4], onlyfiles[8], onlyfiles[12]], ['stock_5','stock_9','stock_13']):
    
    #Read File
    raw = pd.read_csv(p1+f)
    tech = pd.read_csv(p2+f)
    tech = tech.rename(columns={'Unnamed: 0':'Time'})
    raw_df.append(raw)
    tech_df.append(tech)

## Stock 5

In [320]:
stock_5=raw_df[0].merge(tech_df[0],on='Time',how='left')

In [321]:
stock_5

,Time,Open,High,Low,Close,Volume,volumelag,rsilag,fastKlag,fastDlag,ADlag,OBVlag,MA5lag,MA15lag,day5Returnlag,day15Returnlag,PROClag
0,Day1,82.3733,84.6117,82.2390,84.3431,66351500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Day2,84.0297,84.9250,83.3134,83.8954,26750200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Day3,84.1640,85.1489,83.5372,85.0146,40943300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Day4,85.5070,85.9547,82.9105,83.4925,85146500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Day5,83.4925,83.9849,82.8657,83.9402,49108000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,Day752,185.6433,186.6741,183.5816,184.2688,28800748,30188408.0,61.813801,0.809518,0.859436,-0.846190,3.086526e+09,184.60262,180.276507,0.719988,4.829272,-0.580787
752,Day753,185.4960,186.5268,182.8454,183.5816,35874676,28800748.0,58.486122,0.718247,0.820549,-0.555570,3.057725e+09,184.64188,180.770633,0.697405,3.960124,-0.610823
753,Day754,183.8271,187.2631,183.8271,186.5759,34185644,35874676.0,56.492599,0.617109,0.714958,-0.600043,3.021850e+09,184.75968,181.192773,-0.266688,2.578120,-0.373631
754,Day755,186.5759,187.6558,184.6615,185.4960,23502760,34185644.0,62.491778,0.891889,0.742415,0.600000,3.056036e+09,185.26036,181.699993,0.052660,4.768480,1.617887


## Variable Creation

### 1. MACD

In [322]:
stock_5['EMA-12']=np.array(0)

## 12day EMA
N=12.0
coeff=2.0/(N+1)
stock_5.loc[0,'EMA-12'] = stock_5.loc[0,'Close']
for i in range(1,len(stock_5)):
    stock_5.loc[i,'EMA-12'] = (stock_5.loc[i-1,'EMA-12']*(1-coeff))+ (stock_5.loc[i,'Close']*coeff)

stock_5['EMA-26']=np.array(0)
## 26day EMA
N=26.0
coeff=2.0/(N+1)
stock_5.loc[0,'EMA-26'] = stock_5.loc[0,'Close']
for i in range(1,len(stock_5)):
    stock_5.loc[i,'EMA-26'] = (stock_5.loc[i-1,'EMA-26']*(1-coeff))+ (stock_5.loc[i,'Close']*coeff)

##DIF
stock_5['DIF'] = stock_5['EMA-12']-stock_5['EMA-26']

## MACD
N=9
coeff=2.0/(N+1)
stock_5['MACD']=np.array(0)
stock_5.loc[0,'MACD'] = 0
for i in range(2,len(stock_5)):
    stock_5.loc[i,'MACD']= (stock_5.loc[i-1,'MACD']*(1-coeff))+ (stock_5.loc[i,'DIF']*coeff)

## Drop columns
stock_5.drop(['EMA-12','EMA-26','DIF'],axis=1,inplace=True)

### 2. 3-Day Return & MA3 

In [323]:
## 3-Day Return
price = stock_5["Close"]
day3Return = np.zeros(len(stock_5))
for i in range(14,len(stock_5)):
    day3Return[i] = 100*(price[i]-price[i-2])/price[i-2] 

day3Returnlag = np.append(np.array([0]),day3Return[0:len(day3Return)-1])
stock_5['day3Returnlag']=day3Returnlag

## MA3
MA3 = np.zeros(len(stock_5))
for i in range(14,len(stock_5)):
    MA3[i] = stock_5.loc[i-2:i+1,'Close'].sum()/3

MA3lag = np.append(np.array([0]),MA3[0:len(MA3)-1])
stock_5['MA3lag']=MA3lag

### 3. Open,High,Low,Close lag for 5 days.

In [324]:
for i in range(1,6):
    Openlag = np.append(np.zeros(i),stock_5['Open'][0:len(stock_5)-i])
    stock_5['Openlag_{}'.format(i)]=Openlag

    Highlag = np.append(np.zeros(i),stock_5['High'][0:len(stock_5)-i])
    stock_5['Highlag_{}'.format(i)]=Highlag

    Lowlag = np.append(np.zeros(i),stock_5['Low'][0:len(stock_5)-i])
    stock_5['Lowlag_{}'.format(i)]=Lowlag

    Closelag = np.append(np.zeros(i),stock_5['Close'][0:len(stock_5)-i])
    stock_5['Closelag_{}'.format(i)]=Closelag

In [325]:
stock_5.head()

,Time,Open,High,Low,Close,Volume,volumelag,rsilag,fastKlag,fastDlag,...,Lowlag_3,Closelag_3,Openlag_4,Highlag_4,Lowlag_4,Closelag_4,Openlag_5,Highlag_5,Lowlag_5,Closelag_5
0,Day1,82.3733,84.6117,82.2390,84.3431,66351500,NaN,NaN,NaN,NaN,...,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0
1,Day2,84.0297,84.9250,83.3134,83.8954,26750200,NaN,NaN,NaN,NaN,...,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0
2,Day3,84.1640,85.1489,83.5372,85.0146,40943300,NaN,NaN,NaN,NaN,...,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0
3,Day4,85.5070,85.9547,82.9105,83.4925,85146500,NaN,NaN,NaN,NaN,...,82.2390,84.3431,0.0000,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0
4,Day5,83.4925,83.9849,82.8657,83.9402,49108000,NaN,NaN,NaN,NaN,...,83.3134,83.8954,82.3733,84.6117,82.239,84.3431,0.0,0.0,0.0,0.0


### 4. Actual return rate

In [326]:
# stock_5.columns.get_loc('Close')

In [327]:
temp=(stock_5.iloc[1:,stock_5.columns.get_loc('Close')].to_numpy()-stock_5.iloc[:-1,stock_5.columns.get_loc('Close')].to_numpy())/stock_5.iloc[:-1,stock_5.columns.get_loc('Close')].to_numpy()

In [328]:
stock_5['Act_Return'] = np.append([0],temp)

In [329]:
stock_5.head()

,Time,Open,High,Low,Close,Volume,volumelag,rsilag,fastKlag,fastDlag,...,Closelag_3,Openlag_4,Highlag_4,Lowlag_4,Closelag_4,Openlag_5,Highlag_5,Lowlag_5,Closelag_5,Act_Return
0,Day1,82.3733,84.6117,82.2390,84.3431,66351500,NaN,NaN,NaN,NaN,...,0.0000,0.0000,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0,0.000000
1,Day2,84.0297,84.9250,83.3134,83.8954,26750200,NaN,NaN,NaN,NaN,...,0.0000,0.0000,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0,-0.005308
2,Day3,84.1640,85.1489,83.5372,85.0146,40943300,NaN,NaN,NaN,NaN,...,0.0000,0.0000,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0,0.013340
3,Day4,85.5070,85.9547,82.9105,83.4925,85146500,NaN,NaN,NaN,NaN,...,84.3431,0.0000,0.0000,0.000,0.0000,0.0,0.0,0.0,0.0,-0.017904
4,Day5,83.4925,83.9849,82.8657,83.9402,49108000,NaN,NaN,NaN,NaN,...,83.8954,82.3733,84.6117,82.239,84.3431,0.0,0.0,0.0,0.0,0.005362


### 5. Add Y label

In [330]:
stock_5['5day_return'] = np.zeros(len(stock_5))
stock_5['5days_increase'] = np.zeros(len(stock_5))
for i in range(len(stock_5)-5):
    stock_5.loc[i,'5day_return'] =np.where(((stock_5.iloc[i+5,stock_5.columns.get_loc('Close')]-stock_5.iloc[i,stock_5.columns.get_loc('Close')])/\
             stock_5.iloc[i,stock_5.columns.get_loc('Close')])>=0.05,1,0)
    stock_5.loc[i,'5days_increase'] = np.where((stock_5.iloc[i+1:i+5,stock_5.columns.get_loc('Act_Return')]>0).sum()>=3,1,0)

In [331]:
stock_5['Y'] = np.zeros(len(stock_5))
stock_5.loc[(stock_5['5day_return']==1)&(stock_5['5days_increase']==1),'Y']=1

In [332]:
(stock_5['Y']==1).sum()

71

## Drop Columns

In [333]:
list(stock_5.columns)

['Time',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'volumelag',
 'rsilag',
 'fastKlag',
 'fastDlag',
 'ADlag',
 'OBVlag',
 'MA5lag',
 'MA15lag',
 'day5Returnlag',
 'day15Returnlag',
 'PROClag',
 'MACD',
 'day3Returnlag',
 'MA3lag',
 'Openlag_1',
 'Highlag_1',
 'Lowlag_1',
 'Closelag_1',
 'Openlag_2',
 'Highlag_2',
 'Lowlag_2',
 'Closelag_2',
 'Openlag_3',
 'Highlag_3',
 'Lowlag_3',
 'Closelag_3',
 'Openlag_4',
 'Highlag_4',
 'Lowlag_4',
 'Closelag_4',
 'Openlag_5',
 'Highlag_5',
 'Lowlag_5',
 'Closelag_5',
 'Act_Return',
 '5day_return',
 '5days_increase',
 'Y']

In [334]:
stock_5.drop(['Open',
 'High',
 'Low',
 'Close',
 'Volume','Act_Return',
 '5day_return',
 '5days_increase'],axis=1,inplace=True)

In [335]:
stock_5=stock_5.iloc[16:,:]

In [336]:
stock_5.head()

,Time,volumelag,rsilag,fastKlag,fastDlag,ADlag,OBVlag,MA5lag,MA15lag,day5Returnlag,...,Closelag_3,Openlag_4,Highlag_4,Lowlag_4,Closelag_4,Openlag_5,Highlag_5,Lowlag_5,Closelag_5,Y
16,Day17,32780700.0,49.479214,0.411340,0.527182,-0.882399,93654300.0,84.97876,83.999847,0.319819,...,86.1786,83.9849,86.5815,83.9402,85.9547,83.5820,84.5221,83.3582,83.9849,0.0
17,Day18,36612500.0,38.521372,0.027590,0.297607,-0.849033,57041800.0,84.51318,83.850620,-4.999959,...,84.5221,85.7756,87.9693,84.6117,86.1786,83.9849,86.5815,83.9402,85.9547,0.0
18,Day19,24216800.0,37.593856,0.039215,0.159382,-0.571440,32825000.0,83.59992,83.608873,-5.558457,...,84.2535,85.4623,85.9547,84.2088,84.5221,85.7756,87.9693,84.6117,86.1786,0.0
19,Day20,24624500.0,35.020706,0.046506,0.037770,-0.360079,8200500.0,82.48966,83.417860,-4.608026,...,81.6570,84.1640,85.6861,84.1640,84.2535,85.4623,85.9547,84.2088,84.5221,0.0
20,Day21,29800200.0,31.971119,0.046152,0.043958,-0.379342,-21599700.0,81.51372,83.131340,-5.472888,...,81.3884,83.8506,83.8506,81.4779,81.6570,84.1640,85.6861,84.1640,84.2535,0.0


## Train, Test, OOT, Predict Split

In [337]:
stock_5_tt=stock_5.iloc[:388,:].copy()
stock_5_oot=stock_5.iloc[388:488,:].copy()
# stock_5_to_predict=stock_5.iloc[504:,:].copy()

In [338]:
stock_5_oot.tail()

,Time,volumelag,rsilag,fastKlag,fastDlag,ADlag,OBVlag,MA5lag,MA15lag,day5Returnlag,...,Closelag_3,Openlag_4,Highlag_4,Lowlag_4,Closelag_4,Openlag_5,Highlag_5,Lowlag_5,Closelag_5,Y
499,Day500,37338024.0,51.076239,0.589886,0.803368,-0.351351,2.196906e+09,139.43474,136.748340,1.225901,...,141.3468,140.9644,141.6336,138.2875,140.8210,135.0849,136.6145,134.8459,136.4711,0.0
500,Day501,30522872.0,47.929822,0.410114,0.617975,-0.848485,2.166383e+09,139.46342,136.780207,-2.987125,...,140.3907,140.6297,141.6336,140.1039,141.3468,140.9644,141.6336,138.2875,140.8210,0.0
501,Day502,26698642.0,45.658070,0.275286,0.425095,-0.142857,2.139684e+09,138.39268,136.585820,-4.159627,...,138.1441,139.3869,141.3468,138.6221,140.3907,140.6297,141.6336,140.1039,141.3468,0.0
502,Day503,22729012.0,47.554284,0.370790,0.352063,0.703704,2.162413e+09,137.37930,136.614500,-2.928114,...,136.6145,138.1441,139.3391,137.5705,138.1441,139.3869,141.3468,138.6221,140.3907,0.0
503,Day504,20209392.0,47.034439,0.342700,0.329592,-0.830508,2.142204e+09,136.50934,136.633620,-1.522468,...,135.4673,139.0523,139.5303,136.3755,136.6145,138.1441,139.3391,137.5705,138.1441,0.0


In [339]:
stock_5_tt_time=stock_5_tt['Time']
stock_5_oot_time=stock_5_oot['Time']
stock_5_tt_y=stock_5_tt['Y']
stock_5_oot_y=stock_5_oot['Y']

stock_5_tt.drop(['Time','Y'],inplace=True,axis=1)
stock_5_oot.drop(['Time','Y'],inplace=True,axis=1)

for key in stock_5_tt.keys():
    ss = StandardScaler()
    stock_5_tt[key] = ss.fit_transform(stock_5_tt[key].values.reshape(-1, 1))

for key in stock_5_oot.keys():
    stock_5_oot[key] = ss.transform(stock_5_oot[key].values.reshape(-1, 1))

stock_5_tt['Time']=stock_5_tt_time
stock_5_oot['Time']=stock_5_oot_time
stock_5_tt['Y']=stock_5_tt_y
stock_5_oot['Y']=stock_5_oot_y

## Oversampling tnt data

In [340]:
from imblearn.over_sampling import SMOTE 

In [343]:
stock_5_tt.reset_index(inplace=True, drop=True)

In [344]:
stock_5_tt.tail()

,volumelag,rsilag,fastKlag,fastDlag,ADlag,OBVlag,MA5lag,MA15lag,day5Returnlag,day15Returnlag,...,Openlag_4,Highlag_4,Lowlag_4,Closelag_4,Openlag_5,Highlag_5,Lowlag_5,Closelag_5,Time,Y
383,-0.714482,-0.824504,-0.213873,-0.145572,0.956065,0.690534,0.249252,0.308396,1.153624,-1.415155,...,0.090893,0.154410,0.048011,0.153934,0.030623,0.028783,0.044070,0.077542,Day400,0.0
384,-0.116301,-0.580085,0.071545,-0.107338,0.284406,0.759504,0.308103,0.284693,1.013508,-1.234327,...,0.193961,0.320071,0.258943,0.378055,0.095134,0.158638,0.052324,0.158116,Day401,0.0
385,-0.987308,-0.416576,1.170873,0.370559,1.055920,0.795803,0.358677,0.264035,0.015532,-0.591917,...,0.398071,0.335847,0.374749,0.347772,0.197954,0.323910,0.262762,0.381713,Day402,0.0
386,-0.547581,-0.846185,0.469892,0.616630,-1.397507,0.743010,0.340690,0.248381,-0.431908,-0.485417,...,0.210127,0.274712,0.258943,0.307386,0.401571,0.339649,0.378296,0.351500,Day403,1.0
387,-0.821587,-1.056486,0.068700,0.615606,-0.048578,0.700494,0.316213,0.238922,-0.549955,-0.329222,...,0.323563,0.377756,0.327503,0.365768,0.214080,0.278657,0.262762,0.311209,Day404,1.0


In [345]:
X_over = stock_5_tt.iloc[:,:-2]
y_over = stock_5_tt.iloc[:,-1]

In [346]:
sm = SMOTE(sampling_strategy='auto', k_neighbors=3)
X_res, y_res = sm.fit_resample(X_over, y_over)

In [347]:
stock_5_tt_over = X_res.copy()

In [348]:
stock_5_tt_over['Y']=y_res

In [351]:
stock_5_tt_over.shape

(706, 35)

In [352]:
X=stock_5_tt_over.iloc[:,:-1].values
y=stock_5_tt_over.iloc[:,-1]

## Logistic Regression

In [353]:
lg=LogisticRegressionCV(cv=5)
lg.fit(X,y)

LogisticRegressionCV(Cs=10, class_weight=None, cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [354]:
lg.score(X,y)

0.8045325779036827

In [355]:
stock_5_oot

,volumelag,rsilag,fastKlag,fastDlag,ADlag,OBVlag,MA5lag,MA15lag,day5Returnlag,day15Returnlag,...,Openlag_4,Highlag_4,Lowlag_4,Closelag_4,Openlag_5,Highlag_5,Lowlag_5,Closelag_5,Time,Y
404,1.446267e+06,-3.271149,-5.024685,-5.022176,-5.081075,7.032315e+07,0.302892,0.222893,-5.172432,-5.002542,...,0.419376,0.423536,0.354894,0.408976,0.319534,0.454737,0.257132,0.369455,Day405,1.0
405,1.720118e+06,-3.103527,-5.019286,-5.022449,-5.063834,7.204327e+07,0.285420,0.231024,-5.148295,-5.044555,...,0.396496,0.481778,0.273772,0.292493,0.419376,0.423536,0.354894,0.408976,Day406,0.0
406,4.398978e+06,-2.448362,-5.004444,-5.016138,-5.006553,7.644226e+07,0.333262,0.255638,-4.757218,-4.562534,...,0.257132,0.292493,0.173925,0.230091,0.396496,0.481778,0.273772,0.292493,Day407,0.0
407,4.021332e+06,-2.318657,-5.006738,-5.010156,-5.025154,8.046359e+07,0.424369,0.300337,-4.620003,-4.434827,...,0.271692,0.348654,0.196806,0.213446,0.257132,0.292493,0.173925,0.230091,Day408,0.0
408,3.022339e+06,-2.367591,-5.011898,-5.007693,-5.075404,7.744125e+07,0.523381,0.346465,-4.623810,-4.420869,...,0.273772,0.338254,0.257132,0.282092,0.271692,0.348654,0.196806,0.213446,Day409,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,1.624791e+06,-2.824471,-5.021428,-5.012138,-5.062386,9.560020e+07,1.020523,0.903622,-4.993751,-4.982924,...,1.087088,1.116209,0.970600,1.080848,0.831236,0.897798,0.820836,0.891558,Day500,0.0
500,1.328223e+06,-2.961390,-5.029251,-5.020205,-5.084020,9.427197e+07,1.021771,0.905009,-5.177084,-5.102712,...,1.072523,1.116209,1.049642,1.103728,1.087088,1.116209,0.970600,1.080848,Day501,0.0
501,1.161809e+06,-3.060248,-5.035118,-5.028599,-5.053314,9.311016e+07,0.975177,0.896550,-5.228107,-5.059346,...,1.018441,1.103728,0.985161,1.062123,1.072523,1.116209,1.049642,1.103728,Day502,0.0
502,9.890671e+05,-2.977732,-5.030962,-5.031777,-5.016475,9.409923e+07,0.931079,0.897798,-5.174516,-5.030243,...,0.964360,1.016361,0.939399,0.964360,1.018441,1.103728,0.985161,1.062123,Day503,0.0


In [356]:
oot_x=stock_5_oot.iloc[:,:-2].values
oot_y=stock_5_oot.iloc[:,-1]

In [357]:
stock_5_oot[stock_5_oot['Y'] == 1].shape

(13, 36)

In [358]:
lg.score(oot_x,oot_y)

0.13

In [120]:
# pip install imbalanced-learn

## Boosted Tree

### ROC-AUC

In [359]:
for i in [0.0001,0.0005,0.001]:
    oot_score=0
    model = xgb.XGBClassifier(n_estimators=500,learning_rate = i, max_depth = 5, reg_alpha = 20,objective='binary:logistic')
    cv_results = cross_validate(model,X,y, cv=5,\
                    scoring='roc_auc',return_train_score=True)
    print(f'i={i},test_score :{cv_results["test_score"].mean()},train_score:{cv_results["train_score"].mean()}')
    for j in range(5):
        model.fit(X,y,verbose=0)
        oot_score +=roc_auc_score(oot_y,model.predict(oot_x))
    print(f'oot_score:{oot_score/5}')

i=0.0001,test_score :0.7327732592739536,train_score:0.8750687171710547
oot_score:0.47126436781609193
i=0.0005,test_score :0.7508705755660724,train_score:0.9258656541759157
oot_score:0.47126436781609193
i=0.001,test_score :0.7304707115935046,train_score:0.943971062450556
oot_score:0.47126436781609193


## Boosted Tree--TO DECTED TOP 25% 

In [391]:
def my_custom_fdr_func(estimator, X, y_true):
    y_pred_proba = estimator.predict_proba(X)[:,1]
    dff = pd.DataFrame({'y_upward':y_true,'y_pred':y_pred_proba})
    topRows=int(round(len(y_true)*0.25))
    temp=dff.sort_values('y_pred',ascending=False).reset_index()
    score=temp.loc[:topRows,'y_upward'].sum()/temp['y_upward'].sum()-(topRows-temp.loc[:topRows,'y_upward'].sum())/topRows
    print(temp[temp['y_upward']==1].tail(1)['y_pred'])
    return float(score)

In [392]:
oot_score=0
model = xgb.XGBClassifier(n_estimators=300,learning_rate = 0.001, max_depth = 10, reg_alpha = 20,objective='binary:logistic')
cv_results = cross_validate(model,X,y, cv=5,\
                scoring=my_custom_fdr_func,return_train_score=True)
print(f'test_score :{round(cv_results["test_score"].mean(),4)},train_score:{round(cv_results["train_score"].mean(),4)}')
for j in range(5):
    model.fit(X,y,verbose=0)
    oot_score +=my_custom_fdr_func(model, oot_x, oot_y)
print(f'oot_score:{round(oot_score/5,4)}')

141    0.440622
Name: y_pred, dtype: float32
431    0.48414
Name: y_pred, dtype: float32
124    0.455938
Name: y_pred, dtype: float32
427    0.446782
Name: y_pred, dtype: float32
135    0.5
Name: y_pred, dtype: float32
364    0.466941
Name: y_pred, dtype: float32
112    0.492378
Name: y_pred, dtype: float32
346    0.483856
Name: y_pred, dtype: float32
140    0.468109
Name: y_pred, dtype: float32
466    0.468109
Name: y_pred, dtype: float32
test_score :0.4469,train_score:0.4653
87    0.483182
Name: y_pred, dtype: float32
87    0.483182
Name: y_pred, dtype: float32
87    0.483182
Name: y_pred, dtype: float32
87    0.483182
Name: y_pred, dtype: float32
87    0.483182
Name: y_pred, dtype: float32
oot_score:-0.5323


In [393]:
(0.440622+0.48414 + 0.455938 + 0.446782+0.5 +0.466941 +0.492378 +0.483856 +0.468109+0.468109)/10

0.4706874999999999

In [386]:
 for i in [0.001,0.01,0.1]:
        for j in [3,5,10,15,20]:
            oot_score=0
            model = xgb.XGBClassifier(n_estimators=300,learning_rate = i, max_depth = 10, reg_alpha = j,objective='binary:logistic')
            cv_results = cross_validate(model,X,y, cv=5,\
                            scoring=my_custom_fdr_func,return_train_score=True)
            print(f'i={i},j={j},test_score :{round(cv_results["test_score"].mean(),4)},train_score:{round(cv_results["train_score"].mean(),4)}')
            for j in range(5):
                model.fit(X,y,verbose=0)
                oot_score +=my_custom_fdr_func(model, oot_x, oot_y)
            print(f'oot_score:{round(oot_score/5,4)}')

i=0.001,j=3,test_score :0.1645,train_score:0.495
oot_score:-0.5323
i=0.001,j=5,test_score :0.1731,train_score:0.4865
oot_score:-0.5323
i=0.001,j=10,test_score :0.2499,train_score:0.4716
oot_score:-0.5323
i=0.001,j=15,test_score :0.3871,train_score:0.4738
oot_score:-0.5323
i=0.001,j=20,test_score :0.4469,train_score:0.4653
oot_score:-0.5323
i=0.01,j=3,test_score :0.25,train_score:0.5099
oot_score:-0.7662
i=0.01,j=5,test_score :0.2756,train_score:0.5099
oot_score:-0.7662
i=0.01,j=10,test_score :0.3184,train_score:0.5057
oot_score:-0.5323
i=0.01,j=15,test_score :0.3186,train_score:0.5035
oot_score:-0.5323
i=0.01,j=20,test_score :0.2846,train_score:0.5014
oot_score:-0.5323
i=0.1,j=3,test_score :0.327,train_score:0.5099
oot_score:-0.5323
i=0.1,j=5,test_score :0.3099,train_score:0.5099
oot_score:-0.5323
i=0.1,j=10,test_score :0.3014,train_score:0.5099
oot_score:-0.6492
i=0.1,j=15,test_score :0.3187,train_score:0.5057
oot_score:-0.5323
i=0.1,j=20,test_score :0.3015,train_score:0.5014
oot_scor

## Set 0.4707 as the cutoff point

## Random Forest

In [388]:
for i in [200,300]:
    for j in [3,5,10,20]:
        oot_score=0
        model = RandomForestClassifier(n_estimators=i, min_samples_leaf=j,verbose=0)
        cv_results = cross_validate(model,X,y, cv=5,\
                        scoring=my_custom_fdr_func,return_train_score=True)
        print(f'i={i},j={j},test_score :{cv_results["test_score"].mean()},train_score:{cv_results["train_score"].mean()}')
        for m in range(5):
            model.fit(X,y)
            oot_score +=my_custom_fdr_func(model, oot_x, oot_y)
        print(f'i={i},j={j},oot_score:{oot_score/5}')

i=200,j=3,test_score :0.36994187346300017,train_score:0.509926571936947
i=200,j=3,oot_score:-0.48553846153846153
i=200,j=5,test_score :0.3356561591772859,train_score:0.509926571936947
i=200,j=5,oot_score:-0.579076923076923
i=200,j=10,test_score :0.2248647440196736,train_score:0.509926571936947
i=200,j=10,oot_score:-0.48553846153846153
i=200,j=20,test_score :0.12288397048960428,train_score:0.507798912362479
i=200,j=20,oot_score:-0.5089230769230768
i=300,j=3,test_score :0.3869640062597809,train_score:0.509926571936947
i=300,j=3,oot_score:-0.4621538461538462
i=300,j=5,test_score :0.3527990163201431,train_score:0.509926571936947
i=300,j=5,oot_score:-0.5089230769230769
i=300,j=10,test_score :0.2677621283255086,train_score:0.509926571936947
i=300,j=10,oot_score:-0.43876923076923074
i=300,j=20,test_score :0.0717348535658395,train_score:0.507798912362479
i=300,j=20,oot_score:-0.5323076923076923


#### Random Forest is worse than Boosted Tree

#### Neural Net

In [389]:
## hidden_layer_sizes
for i in range(5,10):
    oot_score=0
    model = MLPClassifier(hidden_layer_sizes=(i,), activation='logistic',max_iter=200,alpha=0.001,solver='sgd',\
                          verbose=0,learning_rate_init=0.01,learning_rate='adaptive',tol=0.00001)
    cv_results = cross_validate(model,X,y, cv=5,\
                    scoring=my_custom_fdr_func,return_train_score=True)
    print(f'i={i},test_score :{cv_results["test_score"].mean()},train_score:{cv_results["train_score"].mean()}')
    for m in range(5):
        model.fit(X,y)
        oot_score +=my_custom_fdr_func(model, oot_x, oot_y)
    print(f'oot_score:{oot_score/5}')

i=5,test_score :-0.4962888441761681,train_score:-0.0132295817357091
oot_score:-0.5323076923076923
i=6,test_score :-0.42843505477308297,train_score:0.010167155351727936
oot_score:-0.5323076923076923
i=7,test_score :-0.40331768388106415,train_score:0.02505576021852992
oot_score:-0.5323076923076923
i=8,test_score :-0.40280125195618155,train_score:0.03569907024534495
oot_score:-0.5323076923076923
i=9,test_score :-0.42018108651911473,train_score:0.001659023131092896
oot_score:-0.5323076923076923


In [390]:
## learning_rate
for i in [0.001,0.01,0.1,0.2,0.5]:
    oot_score=0
    model = MLPClassifier(hidden_layer_sizes=(8,), activation='logistic',max_iter=200,alpha=0.001,solver='sgd',\
                          verbose=0,learning_rate_init=i,learning_rate='adaptive',tol=0.00001)
    cv_results = cross_validate(model,X,y, cv=5,\
                    scoring=my_custom_fdr_func,return_train_score=True)
    print(f'i={i},test_score :{cv_results["test_score"].mean()},train_score:{cv_results["train_score"].mean()}')
    for m in range(5):
        model.fit(X,y)
        oot_score +=my_custom_fdr_func(model, oot_x, oot_y)
    print(f'oot_score:{oot_score/5}')

i=0.001,test_score :-0.419901632014308,train_score:-0.19400546324837734
oot_score:-0.5323076923076923
i=0.01,test_score :-0.4199441090990387,train_score:0.00803448362278526
oot_score:-0.5323076923076923
i=0.1,test_score :0.06380281690140846,train_score:0.4695110643310027
oot_score:-0.5323076923076923
i=0.2,test_score :0.05547283702213281,train_score:0.5056712527880108
oot_score:-0.5323076923076923
i=0.5,test_score :0.15773082942097025,train_score:0.5077989123624789
oot_score:-0.5323076923076923


##### Neural Net is worse than Boosted Tree

### To Finalize Predict -- Boosted Tree

In [397]:
stock_5_final=stock_5.iloc[:488,:].copy()
stock_5_to_predict=stock_5.iloc[488:,:].copy()
stock_5_final

,Time,volumelag,rsilag,fastKlag,fastDlag,ADlag,OBVlag,MA5lag,MA15lag,day5Returnlag,...,Closelag_3,Openlag_4,Highlag_4,Lowlag_4,Closelag_4,Openlag_5,Highlag_5,Lowlag_5,Closelag_5,Y
16,Day17,32780700.0,49.479214,0.411340,0.527182,-0.882399,9.365430e+07,84.97876,83.999847,0.319819,...,86.1786,83.9849,86.5815,83.9402,85.9547,83.5820,84.5221,83.3582,83.9849,0.0
17,Day18,36612500.0,38.521372,0.027590,0.297607,-0.849033,5.704180e+07,84.51318,83.850620,-4.999959,...,84.5221,85.7756,87.9693,84.6117,86.1786,83.9849,86.5815,83.9402,85.9547,0.0
18,Day19,24216800.0,37.593856,0.039215,0.159382,-0.571440,3.282500e+07,83.59992,83.608873,-5.558457,...,84.2535,85.4623,85.9547,84.2088,84.5221,85.7756,87.9693,84.6117,86.1786,0.0
19,Day20,24624500.0,35.020706,0.046506,0.037770,-0.360079,8.200500e+06,82.48966,83.417860,-4.608026,...,81.6570,84.1640,85.6861,84.1640,84.2535,85.4623,85.9547,84.2088,84.5221,0.0
20,Day21,29800200.0,31.971119,0.046152,0.043958,-0.379342,-2.159970e+07,81.51372,83.131340,-5.472888,...,81.3884,83.8506,83.8506,81.4779,81.6570,84.1640,85.6861,84.1640,84.2535,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Day500,37338024.0,51.076239,0.589886,0.803368,-0.351351,2.196906e+09,139.43474,136.748340,1.225901,...,141.3468,140.9644,141.6336,138.2875,140.8210,135.0849,136.6145,134.8459,136.4711,0.0
500,Day501,30522872.0,47.929822,0.410114,0.617975,-0.848485,2.166383e+09,139.46342,136.780207,-2.987125,...,140.3907,140.6297,141.6336,140.1039,141.3468,140.9644,141.6336,138.2875,140.8210,0.0
501,Day502,26698642.0,45.658070,0.275286,0.425095,-0.142857,2.139684e+09,138.39268,136.585820,-4.159627,...,138.1441,139.3869,141.3468,138.6221,140.3907,140.6297,141.6336,140.1039,141.3468,0.0
502,Day503,22729012.0,47.554284,0.370790,0.352063,0.703704,2.162413e+09,137.37930,136.614500,-2.928114,...,136.6145,138.1441,139.3391,137.5705,138.1441,139.3869,141.3468,138.6221,140.3907,0.0


In [398]:
# stock_5_final=stock_5.iloc[:504,:].copy()

stock_5_final_time=stock_5_final['Time']
stock_5_to_predict_time=stock_5_to_predict['Time']
stock_5_final_y=stock_5_final['Y']
stock_5_to_predict_y=stock_5_to_predict['Y']

stock_5_final.drop(['Time','Y'],inplace=True,axis=1)
stock_5_to_predict.drop(['Time','Y'],inplace=True,axis=1)

for key in stock_5_final.keys():
    ss = StandardScaler()
    ss.fit(stock_5_final[key].values.reshape(-1,1))
    stock_5_to_predict[key] = ss.transform(stock_5_to_predict[key].values.reshape(-1, 1))

stock_5_final['Time']=stock_5_final_time
stock_5_to_predict['Time']=stock_5_to_predict_time
stock_5_final['Y']=stock_5_final_y
stock_5_to_predict['Y']=stock_5_to_predict_y

In [399]:
stock_5_to_predict

,volumelag,rsilag,fastKlag,fastDlag,ADlag,OBVlag,MA5lag,MA15lag,day5Returnlag,day15Returnlag,...,Openlag_4,Highlag_4,Lowlag_4,Closelag_4,Openlag_5,Highlag_5,Lowlag_5,Closelag_5,Time,Y
504,0.246974,-0.287024,-0.081266,-0.478499,0.092840,1.251026,0.740770,0.782257,0.039813,0.259038,...,0.697709,0.648719,0.764399,0.699956,0.872658,0.798469,0.836690,0.755339,Day505,0.0
505,-0.374407,-0.711655,-0.930496,-0.610469,-0.601444,1.193344,0.729845,0.788791,-0.133892,-0.230119,...,0.708565,0.659330,0.777737,0.736870,0.701468,0.652342,0.767913,0.703338,Day506,0.0
506,-0.726300,-0.750894,-1.354340,-0.859467,-0.047895,1.148244,0.727661,0.789082,-0.336891,-0.085535,...,0.849700,0.754832,0.806637,0.726013,0.712303,0.662931,0.781224,0.740172,Day507,0.0
507,-0.307739,-1.114403,-1.779769,-1.475343,-1.199902,1.088178,0.701440,0.787920,-0.656937,-0.485879,...,0.747649,0.769688,0.806637,0.788982,0.853156,0.758231,0.810066,0.729338,Day508,0.0
508,1.373879,-1.953979,-1.676855,-1.745972,-1.240454,0.967984,0.627588,0.765996,-2.022747,-1.175126,...,0.756334,0.672064,0.759953,0.697785,0.751308,0.773056,0.810066,0.792172,Day509,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,-0.764465,0.771521,1.012534,1.295351,-1.443258,2.539193,2.945675,2.764193,0.074322,0.546434,...,2.858533,2.768414,2.960272,2.858847,2.742260,2.771910,2.865981,2.906531,Day752,0.0
752,-0.820688,0.462367,0.684030,1.143103,-0.933932,2.497469,2.947469,2.786707,0.068096,0.400249,...,2.858533,2.849051,2.914611,2.907902,2.857975,2.767559,2.959394,2.857581,Day753,0.0
753,-0.534077,0.277161,0.320014,0.729692,-1.011874,2.445496,2.952854,2.805940,-0.197685,0.167806,...,2.925420,2.914433,3.062998,3.017161,2.857975,2.848026,2.913825,2.906531,Day754,0.0
754,-0.602511,0.834507,1.309001,0.837191,1.091263,2.495022,2.975740,2.829051,-0.109647,0.536209,...,3.025759,2.936228,3.104092,2.968106,2.924730,2.913269,3.061915,3.015554,Day755,0.0


In [400]:
## Combine TnT & OOT data
X_final = np.append(X,oot_x,axis=0)
y_final = np.append(y,oot_y)

X_to_predict=stock_5_to_predict.iloc[:,:-2].values

model = xgb.XGBClassifier(n_estimators=300,learning_rate = 0.001, max_depth = 10, reg_alpha = 20,objective='binary:logistic')

model.fit(X_final,y_final,verbose=0)
y_pred = model.predict_proba(X_to_predict)[:,1]

In [401]:
y_pred

array([0.42160952, 0.42160952, 0.42160952, 0.42160952, 0.42160952,
       0.42160952, 0.42160952, 0.42160952, 0.42160952, 0.42160952,
       0.42160952, 0.42160952, 0.42160952, 0.42160952, 0.42160952,
       0.42160952, 0.42160952, 0.42160952, 0.42160952, 0.44584852,
       0.44584852, 0.44584852, 0.44584852, 0.44584852, 0.44584852,
       0.44584852, 0.44584852, 0.44584852, 0.44584852, 0.492793  ,
       0.492793  , 0.5       , 0.5       , 0.49980053, 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 0.49980053,
       0.5       , 0.5       , 0.5       , 0.5       , 0.49980053,
       0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 0.49980053,
       0.5       , 0.5       , 0.49980053, 0.5       , 0.5       ,
       0.5       , 0.5       , 0.49980053, 0.5       , 0.5    

In [402]:
stock_5_to_predict['y_pred']=y_pred 

In [277]:
stock_5_buyday=stock_5_to_predict.loc[stock_5_to_predict['y_pred']>=0.41,'Time']

In [280]:
stock_5_buyday.values

array(['Day538', 'Day539', 'Day540', 'Day541', 'Day542', 'Day544',
       'Day546', 'Day553', 'Day554', 'Day555', 'Day556', 'Day557',
       'Day558', 'Day567', 'Day570', 'Day571', 'Day572', 'Day574',
       'Day575', 'Day576', 'Day577', 'Day578', 'Day579', 'Day580',
       'Day581', 'Day582', 'Day583', 'Day584', 'Day586', 'Day587',
       'Day588', 'Day589', 'Day623', 'Day624', 'Day625', 'Day627',
       'Day629', 'Day631', 'Day632', 'Day634', 'Day636', 'Day643',
       'Day648', 'Day649', 'Day650', 'Day651', 'Day652', 'Day653',
       'Day654', 'Day655', 'Day656', 'Day657', 'Day658', 'Day675',
       'Day686', 'Day706', 'Day707', 'Day708', 'Day709', 'Day710',
       'Day711', 'Day712', 'Day714', 'Day720', 'Day721', 'Day722',
       'Day723', 'Day724', 'Day725', 'Day726', 'Day729', 'Day730',
       'Day731', 'Day754', 'Day755', 'Day756'], dtype=object)

In [403]:
stock_5_to_predict['buy'] = np.where(stock_5_to_predict['y_pred']>=0.4707,1,0)

## Calculate return

In [412]:
stock_5_to_buy=raw_df[0].iloc[504:,:].copy()
stock_5_to_buy['buy']=stock_5_to_predict['buy']

In [413]:
stock_5_to_buy

,Time,Open,High,Low,Close,Volume,buy
504,Day505,136.5667,136.7579,134.7981,135.4195,39815540,0
505,Day506,135.4195,136.2321,134.3201,135.2283,31130374,0
506,Day507,134.0810,135.0849,133.1250,133.4118,41460992,0
507,Day508,133.4596,133.7942,127.0543,127.9626,82965480,0
508,Day509,126.5763,127.1499,122.6089,123.8517,101575504,0
...,...,...,...,...,...,...,...
751,Day752,185.6433,186.6741,183.5816,184.2688,28800748,0
752,Day753,185.4960,186.5268,182.8454,183.5816,35874676,0
753,Day754,183.8271,187.2631,183.8271,186.5759,34185644,1
754,Day755,186.5759,187.6558,184.6615,185.4960,23502760,1


In [426]:
stock_5_to_buy.loc[stock_5_to_buy['buy']==1,'Time'].values

array(['Day534', 'Day535', 'Day536', 'Day537', 'Day538', 'Day539',
       'Day540', 'Day541', 'Day542', 'Day543', 'Day544', 'Day545',
       'Day546', 'Day547', 'Day548', 'Day549', 'Day550', 'Day551',
       'Day552', 'Day553', 'Day554', 'Day555', 'Day556', 'Day557',
       'Day558', 'Day559', 'Day560', 'Day561', 'Day562', 'Day563',
       'Day564', 'Day565', 'Day566', 'Day567', 'Day568', 'Day569',
       'Day570', 'Day571', 'Day572', 'Day573', 'Day574', 'Day575',
       'Day576', 'Day577', 'Day578', 'Day579', 'Day580', 'Day581',
       'Day582', 'Day583', 'Day584', 'Day585', 'Day586', 'Day587',
       'Day588', 'Day589', 'Day590', 'Day591', 'Day592', 'Day593',
       'Day594', 'Day595', 'Day596', 'Day597', 'Day598', 'Day599',
       'Day600', 'Day617', 'Day618', 'Day619', 'Day623', 'Day624',
       'Day625', 'Day626', 'Day627', 'Day628', 'Day629', 'Day630',
       'Day631', 'Day632', 'Day633', 'Day634', 'Day635', 'Day636',
       'Day637', 'Day638', 'Day639', 'Day640', 'Day641', 'Day6

In [428]:
buy_day = 0
hold = 0
capi = 10000

for i in range(len(stock_5_to_buy)):
    
    if i != 251:
        
        if  not hold:
            if stock_5_to_buy.iloc[i,-1]==1:
                buy_day=i
                print(buy_day+505)
                hold=capi/stock_5_to_buy.iloc[i,4]
                capi = 0
                
        else:
            if ((i - buy_day) % 5 == 0) & (stock_5_to_buy.iloc[i,-1]!=1):
                capi += hold*stock_5_to_buy.iloc[i,4]
                hold = 0
                
    # last day
    else:
        capi += hold*stock_5_to_buy.iloc[i,4]
        hold = 0
print(capi)      

534
617
623
675
687
702
751
11862.736918179091
